In [ ]:
import os
import sys
import numpy as np
import cv2 
import pandas as pd
import matplotlib.pyplot as plt 
import random
%matplotlib inline

os.environ['MOMAPS_HOME'] = '/home/labs/hornsteinlab/Collaboration/MOmaps'

sys.path.insert(1, os.getenv('MOMAPS_HOME'))
print(f"MOMAPS_HOME: {os.getenv('MOMAPS_HOME')}")

from cellpose import models
import cellpose
from shapely.geometry import Polygon

from src.common.lib import image_metrics
from src.common.lib.preprocessing_utils import rescale_intensity, handle_img_shape, segment
from src.common.lib.image_sampling_utils import sample_images_all_markers_all_lines, sample_raw_images, sample_processed_images
from src.common.lib.eval_preprocessing_steps_utils import get_processed_images, plot_images, test_cellpose, reconstruct_images

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# INPUT_DIR_BATCH = "/home/labs/hornsteinlab/Collaboration/MOmaps/input/images/raw/SpinningDisk/FUS_lines_stress_2024_sorted/batch1"
# INPUT_DIR_BATCH = "/home/labs/hornsteinlab/Collaboration/MOmaps/input/images/raw/Opera18Days_sorted/batch2"
INPUT_DIR_BATCH = "/home/labs/hornsteinlab/Collaboration/MOmaps/input/images/raw/SpinningDisk/NOVA_d18_neurons_sorted/batch2"
EXPECTED_SITE_WIDTH, EXPECTED_SITE_HEIGHT = 1024, 1024

sample_size = 10
rep = 'rep1'
marker = 'DAPI'
condition = 'Untreated'
cell_line = 'SNCA'
panel = 'panelL'

In [ ]:
images_paths = sample_raw_images(INPUT_DIR_BATCH, marker,
                                 cell_line=cell_line, condition=condition, sample_size=sample_size, rep=rep, panel=panel)
# images_paths = sample_images_all_markers_all_lines(input_dir_batch=INPUT_DIR_BATCH, _sample_size_per_markers=5, _num_markers=26, 
#                                         raw=True, all_conds=True, rep_count=2, cond_count=2, exclude_DAPI=False, markers_to_include=None)
# images_paths = random.sample(images_paths, len(images_paths))
images_paths

In [ ]:
images = []
for p in images_paths:
    image = cv2.imread(p, cv2.IMREAD_ANYDEPTH)
    images.append(image)
    
images = np.stack(images, axis=0)
images.shape

In [ ]:
plot_images(images, images_paths)

In [ ]:
images_processed = np.zeros((images.shape[0], EXPECTED_SITE_WIDTH, EXPECTED_SITE_HEIGHT))
images_paths_processed = images_paths.copy()

# Handle size

In [ ]:
for i in range(len(images)):
    images_processed[i] = handle_img_shape(images[i], EXPECTED_SITE_WIDTH, EXPECTED_SITE_HEIGHT)

In [ ]:
plot_images(images_processed, images_paths_processed, 10)

# Rescale Intensity

In [ ]:
# Apply rescale intensity
for i in range(len(images_processed)):
    images_processed[i] = rescale_intensity(images_processed[i])

In [ ]:
plot_images(images_processed, images_paths_processed, 10, figsize=(35,10))

# Brenner

In [ ]:
brenner_cutoffs = pd.read_csv("/home/labs/hornsteinlab/Collaboration/MOmaps/src/preprocessing/sites_validity_bounds_spd18days.csv", index_col=0)
brenner_cutoffs

brenner_marker = brenner_cutoffs.loc[marker]
brenner_lower_bound, brenner_upper_bound = brenner_marker[0], brenner_marker[1]

In [ ]:
brenner_for_sampled_images = []
n_samples = 3

for i in range(len(images_processed)):
    brenner_for_sampled_images.append(image_metrics.calculate_image_sharpness_brenner(images_processed[i]))
brenner_for_sampled_images = np.asarray(brenner_for_sampled_images)

# Show images passed filter
passed_images_indexes = np.argwhere((brenner_for_sampled_images >= brenner_lower_bound) & (brenner_for_sampled_images <= brenner_upper_bound)).reshape(-1)
if len(passed_images_indexes) > 0:
    plot_images(images_processed[passed_images_indexes], images_paths_processed[passed_images_indexes], n_samples=min(n_samples, len(passed_images_indexes)), suptitle='Valid')
else:
    print("No valid files (in terms of Brenner)")

# Show images failed filter
failed_images_indexes = list(set(np.arange(len(images_processed))) - set(passed_images_indexes))
if len(failed_images_indexes) > 0:
    plot_images(images_processed[failed_images_indexes], images_paths_processed[failed_images_indexes], n_samples=min(n_samples, len(failed_images_indexes)), suptitle='Invalid')
else:
    print("All files are valid (in terms of Brenner)")

images_processed = images_processed[passed_images_indexes]
images_paths_processed = images_paths_processed[passed_images_indexes]


# Cellpose

In [ ]:
# Apply cellpose on remaining images
# NOTE! Use only with marker was set to DAPI!
test_cellpose(images_processed, images_paths_processed)

# Processed images

In [ ]:
# INPUT_DIR_BATCH_PROCESSED = "/home/labs/hornsteinlab/Collaboration/MOmaps/input/images/processed/Opera18Days/batch1"
INPUT_DIR_BATCH_PROCESSED = "/home/labs/hornsteinlab/Collaboration/MOmaps/input/images/processed/spd2/SpinningDisk/spd18days/batch1"

figsize=(20,6)
sample_size = 5
marker='FUS'
cell_line='WT'
condition='Untreated'
rep='rep1'

_ = get_processed_images(INPUT_DIR_BATCH_PROCESSED, sample_size=sample_size, marker=marker, cell_line=cell_line, condition=condition, rep=rep, figsize=figsize)

# Reconstructed images

NOTE: Change env to momaps_torch!

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import torch

%matplotlib inline

os.environ['MOMAPS_HOME'] = '/home/labs/hornsteinlab/Collaboration/MOmaps'

sys.path.insert(1, os.getenv('MOMAPS_HOME'))
print(f"MOMAPS_HOME: {os.getenv('MOMAPS_HOME')}")

from src.common.lib.image_sampling_utils import sample_images_all_markers_all_lines, sample_raw_images, sample_processed_images
from src.common.lib.eval_preprocessing_steps_utils import get_processed_images, reconstruct_images
from src.common.lib.model import Model
from src.common.lib.utils import get_if_exists, load_config_file


from src.common.lib import metrics 

In [ ]:
config_path_model = "./src/models/neuroself/configs/model_config/TLNeuroselfB78NoDSModelConfig"
config_model = load_config_file(config_path_model, 'model')
model = Model(config_model)
model.load_model(num_fc_output_classes=225)

In [ ]:
INPUT_DIR_BATCH_PROCESSED = "/home/labs/hornsteinlab/Collaboration/MOmaps/input/images/processed/spd2/SpinningDisk/spd18days/batch1"#"/home/labs/hornsteinlab/Collaboration/MOmaps/input/images/processed/Opera18Days/batch2"
figsize=(8,6)
sample_size = 10
marker = 'AGO2'
cell_line = 'WT'
condition='Untreated'
rep = 'rep2'

In [ ]:
images, images_paths = get_processed_images(INPUT_DIR_BATCH_PROCESSED, sample_size=sample_size, marker=marker, cell_line=cell_line, condition=condition, rep=rep, figsize=figsize, plot=False)
# images.shape

reconstruct_images(model, images)